In [35]:
import pandas as pd

In [36]:
data = pd.read_csv('Key_Dynamics.csv')

In [37]:
data.drop(["sessionIndex", "rep"], axis=1, inplace=True)

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[0])],remainder='passthrough')
unique_cols = data['subject'].nunique()

data = np.array(ct.fit_transform(data))
X = data[:,unique_cols:]
y = data[:,:unique_cols]
print(y.shape)

(20400, 51)


In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [41]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [42]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=unique_cols, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
510/510 [==============================] - 4s 7ms/step - loss: 1.7649 - accuracy: 0.5920
Epoch 2/10
510/510 [==============================] - 3s 7ms/step - loss: 1.0333 - accuracy: 0.7741
Epoch 3/10
510/510 [==============================] - 3s 7ms/step - loss: 1.0103 - accuracy: 0.7730
Epoch 4/10
510/510 [==============================] - 4s 7ms/step - loss: 0.9597 - accuracy: 0.7878
Epoch 5/10
510/510 [==============================] - 3s 7ms/step - loss: 0.9856 - accuracy: 0.7794
Epoch 6/10
510/510 [==============================] - 3s 7ms/step - loss: 0.9687 - accuracy: 0.7775
Epoch 7/10
510/510 [==============================] - 4s 7ms/step - loss: 1.0002 - accuracy: 0.7724
Epoch 8/10
510/510 [==============================] - 4s 7ms/step - loss: 0.9976 - accuracy: 0.7744
Epoch 9/10
510/510 [==============================] - 4s 7ms/step - loss: 0.9455 - accuracy: 0.7884
Epoch 10/10
510/510 [==============================] - 4s 7ms/step - loss: 0.9524 - accuracy: 0.7833

In [44]:
model.evaluate(X_test, y_test)

128/128 [==============================] - 1s 6ms/step - loss: 0.9174 - accuracy: 0.7775


[0.9173995852470398, 0.7774509787559509]